In [1]:
import boto3
import botocore

BUCKET_NAME = 'ts-dev-cs-training-data'
PREFIX = 'rbp-research/data/e2e/2017-09-11/'

s3 = boto3.resource('s3').Bucket(BUCKET_NAME)

def download_file(k, diskFilename):
    try:
        s3.download_file(k, diskFilename)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [2]:
def get_a_train_file_key(s3, prefix):
    for o in s3.objects.filter(Prefix=prefix, Delimiter='/'):
        yield o.key

gen = get_a_train_file_key(s3, PREFIX)

In [3]:
import json
import os

def get_json_object(filename):
    with open(filename) as json_data:
        return json.load(json_data)

def parse_file(k):
    download_file(k, 'test.json')
    d = get_json_object('test.json')
    os.remove('test.json')
    return d

In [ ]:
def parse_single_file(s):
    columns=['text', 'top', 'bottom', 'left', 'right', 'pageWidth', 'pageHeight']
    j = parse_file(s)
    s = s.split('/')[1].split('.')[0]
    words = []
    for word in j.get('describedWords'):
        f = word.get('features')
        try:
            w = [word.get('text'), word.get('top'), word.get('bottom'), word.get('left'), word.get('right'), float(f.get('pageWidth')), float(f.get('pageHeight'))]
            words.append(w)
        except:
            pass
        
    df = pd.DataFrame(words, columns=columns)
    df['relativeHeight'] = df['bottom'] / df['pageHeight']
    df['relativeWidth'] = df['right'] / df['pageWidth']
    return df

def save_list_of_docs(docs, prefix, suffix):
    import pickle
    filename = prefix + 'document' + str(suffix) + '.pickle'
    pickle.dump(documents, open(filename, 'wb'))
    return filename
    
def load_list_of_docs(f):
    import pickle
    return pickle.load(open(f, 'rb'))
    

In [ ]:
import pandas as pd
import os

FILES_PREFIX = 'dumpData/'

documents = {}
i = 0
for s in gen:
    documents[s] = parse_single_file(s)
    
    if i % 1000 == 0:
        filename = save_list_of_docs(documents, FILES_PREFIX, i)
        print(PREFIX + filename)
        s3.upload_file(filename, PREFIX + filename)
        documents = {}
    
    #if i > 30:
    #    break
    i = i + 1
    

rbp-research/data/e2e/2017-09-11/dumpData/document0.pickle


In [171]:
for f in [f for f in os.listdir(FILES_PREFIX) if not os.path.isdir(f)]:
    d = load_list_of_docs(FILES_PREFIX + f)
    
    print(d)
    break

{'hackathon_2017/00490d5f-cf24-4d02-b168-26b1f7b8c5a1.training.json':                   text   top  bottom  left  right  pageWidth  pageHeight  \
0                 4002  1008    1023   188    213      892.0      1261.0   
1           Bezahlung.   966     981   236    297      892.0      1261.0   
2               Basler  1008    1023    64    101      892.0      1261.0   
3             3.242,02   624     639   739    790      892.0      1261.0   
4                   61   243     258   652    666      892.0      1261.0   
5               460,10   814     829   527    568      892.0      1261.0   
6               SWIFT:  1025    1040   306    348      892.0      1261.0   
7               Anzahl   653     668    69    112      892.0      1261.0   
8             pauschal   715     730   172    224      892.0      1261.0   
9                Kühne   225     240    95    130      892.0      1261.0   
10               Ihren   870     885   432    464      892.0      1261.0   
11              Be